Validação de Modelos com Cross Validation

In [1]:
#Carregando os dados
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [2]:
#Dividindo em x e y
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

Sem Cross Validation

In [3]:
#Dividindo em treino e teste - SEM CV
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [4]:
#Acurácia do Dummy classifier - SEM CV
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 50.96%


C:\Users\Windows 10\anaconda3\lib\site-packages\sklearn\dummy.py:131: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  warnings.warn("The default value of strategy will change from "


In [5]:
#acurácia Arvore de decisão - SEM CV
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


Com Cross validation

In [6]:
def imprime_cv(results):
    media = results['test_score'].mean() 
    desvio_padrao = results['test_score'].std()
    intervalo_conf = ((media - 2*desvio_padrao)*100, (media + 2*desvio_padrao)*100)
    print("Média das acurácias: %.2f" % (media * 100))
    print("Intervalo de confiança: [%.2f , %.2f]" % intervalo_conf)
    

In [7]:
#Acurácia da Arvore de decisão com CV
#Estamos usando KFold por trás dos panos
#sem shuffle
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

SEED = 42
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
cv = KFold(n_splits = 10)
results = cross_validate(modelo, x, y, cv = cv)
#same thing results = cross_validate(modelo, x, y, cv = 10)

print("sem shuffle")
imprime_cv(results)

sem shuffle
Média das acurácias: 75.78
Intervalo de confiança: [74.37 , 77.19]


In [8]:
#Acurácia da Arvore de decisão com CV
#Estamos usando KFold por trás dos panos
#com shuffle/usado quando os dados tem um sequencia lógica
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

SEED = 42
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
cv = KFold(n_splits = 10, shuffle = True)
results = cross_validate(modelo, x, y, cv = cv)
#same thing results = cross_validate(modelo, x, y, cv = 10)

print("com shuffle")
imprime_cv(results)

com shuffle
Média das acurácias: 75.76
Intervalo de confiança: [72.76 , 78.76]


Estratificação

In [9]:
#Resultados horriveis
from sklearn.model_selection import StratifiedKFold

dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_cv(results)

Média das acurácias: 57.84
Intervalo de confiança: [34.29 , 81.39]


In [10]:
#Estratificação é feita em y
from sklearn.model_selection import StratifiedKFold

dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

cv = StratifiedKFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_cv(results)

Média das acurácias: 75.78
Intervalo de confiança: [72.81 , 78.75]


Gerando dados aleatorios

In [28]:
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2,3, size = len(dados))
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio,modelo
0,30941.02,1,18,35085.22134,21,19
1,40557.96,1,20,12622.05362,24,22
2,89627.50,0,12,11440.79806,14,12
3,95276.14,0,3,43167.32682,7,5
4,117384.68,1,4,12770.11290,8,6


In [29]:
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.modelo.unique()

array([23, 26, 16,  9, 10, 14, 20, 21,  7, 15, 18, 19, 17, 13,  5, 12, 24,
       22, 25, 11,  6,  4,  8,  3], dtype=int64)

In [31]:
dados.modelo.value_counts().head()

22    840
21    826
20    801
23    749
18    672
Name: modelo, dtype: int64

Grupo Kfold

In [34]:
#Importante para a IC
#Group não é usado no X
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, groups = dados.modelo, cv = cv, return_train_score=False)
imprime_cv(results)

Média das acurácias: 75.75
Intervalo de confiança: [73.61 , 77.90]


Cross validation com StandardScaler

In [41]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A Acurácia foi de %.2f" % acuracia)

A Acurácia foi de 74.40


In [42]:
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler =StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits = 10)
result = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_cv(results)

Média das acurácias: 75.75
Intervalo de confiança: [73.61 , 77.90]
